In [6]:
import re
import pandas as pd
from docxtpl import DocxTemplate

# Utility to parse a dimension string (e.g. '12m', '12 cm', '5mm') and convert to meters
def parse_to_meters(s: str) -> float:
    s = s.strip().lower().replace(' ', '')
    match = re.match(r"([0-9]*\.?[0-9]+)(m|cm|mm)", s)
    if not match:
        raise ValueError(f"Can't parse dimension: {s}")
    value, unit = match.groups()
    val = float(value)
    if unit == 'm':
        return val
    elif unit == 'cm':
        return val / 100
    elif unit == 'mm':
        return val / 1000
    else:
        raise ValueError(f"Unknown unit: {unit}")

# Load your CSV into a DataFrame
# The CSV should have headers: Column height, Column radius, Column thickness

def load_parameters(csv_path: str) -> dict:
    df = pd.read_csv(csv_path)
    # assume first row contains your data
    row = df.iloc[0]
    height_m = parse_to_meters(row['Column height'])
    radius_m = parse_to_meters(row['Column radius'])
    thickness_m = parse_to_meters(row['Column thickness'])
    # compute area and volume
    area_m2 = 3.141592653589793 * radius_m**2
    volume_m3 = area_m2 * height_m
    return {
        'length': f"{height_m:.3f} m",
        'radius': f"{radius_m:.3f} m",
        'thickness': f"{thickness_m:.3f} m",
        'area': f"{area_m2:.4f} m²",
        'volume': f"{volume_m3:.4f} m³"
    }

# Fill the Word template using docxtpl
# The template docx should use Jinja2 placeholders like {{ length }}, {{ radius }}, etc.

def fill_template(template_path: str, output_path: str, context: dict):
    tpl = DocxTemplate(template_path)
    tpl.render(context)
    tpl.save(output_path)

if __name__ == '__main__':
    # paths can be adjusted
    csv_file = 'input_test_sheet.csv'
    template_file = 'template.docx'
    output_file = 'filled_pillar.docx'

    params = load_parameters(csv_file)
    print("Loaded parameters:", params)

    fill_template(template_file, output_file, params)
    print(f"Generated file: {output_file}")


Loaded parameters: {'length': '12.000 m', 'radius': '1.660 m', 'thickness': '5.000 m', 'area': '8.6570 m²', 'volume': '103.8837 m³'}
Generated file: filled_pillar.docx
